In [ ]:
pip install fasttext
from keras.preprocessing.text import Tokenizer
from keras import regularizers
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.models import Sequential,Dense,LSTM,Embedding,Dropout,load_model
from keras.callbacks import ModelCheckpoint,EarlyStopping,Callback
from keras.optimizers import RMSprop
from keras.metrics import CategoricalCrossentropy
from pickle import dump,load
import numpy as np
import fasttext as ft
from keras.callbacks import Callback
from matplotlib import pyplot as plt
from IPython.display import clear_output

     |████████████████████████████████| 71kB 5.4MB/s 
  Created wheel for fasttext: filename=fasttext-0.9.2-cp36-cp36m-linux_x86_64.whl size=3097719 sha256=31ec72b9c2b2ff571833002b4aa7ec7d75bfe4bf6c20f358b44bbcafef4727e1
  Stored in directory: /root/.cache/pip/wheels/98/ba/7f/b154944a1cf5a8cee91c154b75231136cc3a3321ab0e30f592
Successfully built fasttext


In [ ]:
# Utility function used to plot the model's progresses through the training.

class PlotLearning(Callback):
    def on_train_begin(self, logs={}):
        self.i = 0
        self.x = []
        self.losses = []
        self.val_losses = []
        self.acc = []
        self.val_acc = []
        self.fig = plt.figure()
        
        self.logs = []

    def on_epoch_end(self, epoch, logs={}):
        
        self.logs.append(logs)
        self.x.append(self.i)
        self.losses.append(logs.get('loss'))
        self.val_losses.append(logs.get('val_loss'))
        self.acc.append(logs.get('categorical_crossentropy'))
        self.val_acc.append(logs.get('val_categorical_crossentropy'))
        self.i += 1
        f, (ax1, ax2) = plt.subplots(1, 2, sharex=True)
        
        clear_output(wait=True)
  
        ax1.set_yscale('log')
        ax1.plot(self.x, self.losses, label="loss")
        ax1.plot(self.x, self.val_losses, label="val_loss")
        ax1.legend()
        
        ax2.plot(self.x, self.acc, label="cross entropy")
        ax2.plot(self.x, self.val_acc, label="validation cross entropy")
        ax2.legend()

        plt.show();

def replace_right(source, target, replacement, replacements=None):
    return replacement.join(source.rsplit(target, replacements))

In [ ]:
# Open the file and encode newlines as standalone symbols
with open("DC-poem-formatUTF_withoutBOL.txt", encoding='latin-1') as file:
    text = file.read().lower()
text = text.replace('\n',' _verse_ ')
print('Number of Characters is:', len(text))
ftmodel = ft.train_unsupervised(input='DC-poem-formatUTF_withoutBOL.txt', minn=3, maxn=6, dim=100, epoch=20)

#Initialize parameters
SEQUENCE_LEN = 50 # The length of each sequence used to predict
STEP = 1 # Stride
X = [] # Input Variables 
Y = [] # Output Variables

# Fetch all the words inside the file
words = []
start_token = "<start>"
sentences = [w for w in text.split('_verse_') if w.strip() != '' or w != '\n']
sentences = sentences[1:-1]
print('The length of the text:', len(sentences))
rev_verses = []
for verse in sentences:
  temp = verse.split()
  temp.reverse()
  rev_verses.append(" ".join(temp))
rev_verses = [verse + " _verse_ " if verse != rev_verses[-2] else verse for verse in rev_verses ]
rev_verses = ["<start>"] + rev_verses
# Encode the words using integers
tokenizer = Tokenizer(filters=[], lower=True, oov_token="_unk_")
tokenizer.fit_on_texts(rev_verses)
words_tokenized = tokenizer.texts_to_sequences(rev_verses)
vocab_size = len(tokenizer.word_index) + 1
print('Number of Words is:', vocab_size)
dump(tokenizer, open('generaTerzinaTokenizer.pkl', 'wb'))
# Flatten the resulting List
words_tokenized = [item for sublist in words_tokenized for item in sublist]

#padding_token_val = words_tokenized[0]
start_token_val = words_tokenized[0]

for i in range(1, len(words_tokenized), STEP):
  #do not predict start token
  tmp_sequence = []
  if words_tokenized[i] != start_token_val:
    # get the SEQUENCE_LEN token before the one to predict, avoiding out of bound access and starting from the start token
    lower_bound = i-(SEQUENCE_LEN) if i-(SEQUENCE_LEN) >= 0 else 0
    tmp_sequence = words_tokenized[lower_bound:i]
    try:
      index = tmp_sequence.index(start_token_val)
      tmp_sequence = tmp_sequence[index:]
    except:
      pass 
    tmp_sequence = pad_sequences([tmp_sequence], maxlen=SEQUENCE_LEN, truncating='pre', padding='pre',value=0)[0]
    tmp_sequence = [x for x in tmp_sequence]
    X.append(tmp_sequence)
    Y.append(words_tokenized[i])
print('Number of Sequences:', len(X), len(Y))

X = np.array(X)
Y = np.array(Y)
Y = to_categorical(Y, num_classes=vocab_size)
print(X[0])
print(np.argmax(Y[0]))


Number of Characters is: 704819
The length of the text: 19146
['<start>', 'vita nostra di cammin del mezzo nel _verse_ ', 'oscura selva una per ritrovai mi _verse_ ', 'smarrita era via diritta la ché _verse_ ', '<tercet>', 'dura cosa è era qual dir a quanto ahi _verse_ ', 'forte e aspra e selvaggia selva esta _verse_ ', 'paura la rinova pensier nel che _verse_ ', '<tercet>', "morte più è poco che amara è tant' _verse_ ", "trovai vi ch'i' ben del trattar per ma _verse_ ", "scorte v'ho ch'i' cose l'altre de dirò _verse_ ", '<tercet>', "v'intrai i' com' ridir ben so non io _verse_ ", "punto quel a sonno di pien era tant' _verse_ ", 'abbandonai via verace la che _verse_ ', '<tercet>', "giunto colle d'un piè al fui ch'i' poi ma _verse_ ", 'valle quella terminava dove là _verse_ ', "compunto cor il paura di m'avea che _verse_ ", '<tercet>', 'spalle sue le vidi e alto in guardai _verse_ ', "pianeta del raggi de' già vestite _verse_ ", 'calle ogne per altrui dritto mena che _verse_ ', '<tercet

In [ ]:
def build_embedding_matrix(vocabulary,embedding_dimension):
  numWords = len(vocabulary.word_index) + 1
  embedding = np.empty(shape=(numWords,embedding_dimension))
  for word in vocabulary.word_index:
      embedding[vocabulary.word_index[word]] = ftmodel.get_word_vector(word)
  return embedding

embedding_dimension = 100
embedding_matrix = build_embedding_matrix(tokenizer,embedding_dimension)

In [ ]:
import tensorflow as tf

# Create the neural network

model_filepath = "weights-{epoch:02d}-{loss:.4f}.hdf5"
model = Sequential()
model.add(Embedding(vocab_size, embedding_dimension, input_length=X.shape[1], mask_zero=True, weights = [embedding_matrix]))
model.add(LSTM(256, return_sequences=True))
model.add(Dropout(0.2))
model.add(Dense(vocab_size, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(), metrics=['categorical_crossentropy'])
model.summary()
# Define Callbacks
es = EarlyStopping(monitor='categorical_crossentropy', mode='min', verbose=2, patience=150)
ck = ModelCheckpoint(model_filepath, monitor='categorical_crossentropy', verbose=1, mode='min')
# Start Training
model.fit(X, Y, batch_size=128, shuffle = False, epochs=300, callbacks=[es,ck,PlotLearning()])
model.save('model-last.h5')

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 50, 100)           1378100   
_________________________________________________________________
lstm_2 (LSTM)                (None, 50, 256)           365568    
_________________________________________________________________
dropout_2 (Dropout)          (None, 50, 256)           0         
_________________________________________________________________
dense_2 (Dense)              (None, 50, 13781)         3541717   
Total params: 5,285,385
Trainable params: 5,285,385
Non-trainable params: 0
_________________________________________________________________
None


OSError: ignored

In [ ]:

def sample(preds, temperature=0.01):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)
    
generated = []
to_print = []
n_words = 1000
predicted_word = ''
#tokenizer = load(open('/content/generaTerzinaTokenizer.pkl', 'rb'))
# Load pre-trained data
#model = load_model('/content/weights-55-5.0572.hdf5')
genera_terzina_input = ['<start> vita nostra di cammin del mezzo nel _verse_ oscura selva una per ritrovai mi _verse_ smarrita era via diritta la ché _verse_ <tercet>']
genera_terzina_input = tokenizer.texts_to_sequences(genera_terzina_input)[0]
genera_terzina_input = [0] * (50 - len(genera_terzina_input)) + genera_terzina_input
text_encoded = genera_terzina_input
for i in range(n_words):
    # Fix the length of the input sequence
    text_encoded = np.array(text_encoded[-50:]).reshape(1,50)
    # Get the model output
    prediction = model.predict([text_encoded], verbose=0)[0]
    # Sample and encode the output
    val = sample(prediction)
    predicted_word = tokenizer.index_word[val]
    generated.append(predicted_word)
    text_encoded = np.append(text_encoded, val)
    
    

In [ ]:

generated = ['\n' if x=='_verse_' or x=='_end_' or x=='<tercet>' else x for x in generated]
copy = generated.copy()
while copy :
   line = ""
   while copy[0] is not '\n':
    line = copy[0] +" " + line
    copy.pop(0)
   copy.pop(0)
   print(line)